In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [16]:
skip_training = False  

In [17]:
data_dir = tools.select_data_dir()

The data directory is ../data


In [18]:
device = torch.device('cpu')

In [19]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the movie (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5.

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [20]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [21]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

In [22]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """

        super(RecommenderSystem, self).__init__()
        self.user_factors = torch.nn.Embedding(n_users + 1, 150)
        self.item_factors = torch.nn.Embedding(n_items + 1, 150)
        
        self.linear1 = torch.nn.Linear(300, 100)
        self.linear2 = torch.nn.Linear(100, 200)
        self.linear3 = torch.nn.Linear(200, 300)
        self.linear4 = torch.nn.Linear(300, 1)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()

        self.drop1 = nn.Dropout(0.25, inplace=False)
        self.drop2 = nn.Dropout(0.5, inplace=False)


        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        users = self.user_factors(user_ids)
        items = self.item_factors(item_ids)
        x = torch.cat([users, items], 1)
        
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        
        x = self.linear3(x)
        x = self.relu3(x)
        
        x = self.linear4(x)
 
        output_scores = x.reshape(user_ids.shape,)
        return output_scores

In [23]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


## Train the model

In [24]:
# Create the model
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [25]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)   

if not skip_training:
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.95)
    epochs = 5
    total_loss = 0.0
    
    for epoch in range(epochs):
        print('Epoch: {}'.format(epoch))
        for idx, (train_x1, train_x2, train_label) in enumerate(trainloader):
            scheduler.step()
            train_x1 = torch.tensor(train_x1).to(torch.int64)
            train_x2 = torch.tensor(train_x2).to(torch.int64)
            optimizer.zero_grad()
            predict_y = model.forward(train_x1, train_x2)
            loss = criterion(predict_y, train_label.float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if idx % 150 == 0:
                print('idx: {:<4}, loss: {:.4f}, total loss: {:.3f}'.format(idx, loss, total_loss))

Epoch: 0
idx: 0   , loss: 12.3753, total loss: 12.375


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
<ipython-input-25-3b33b724455d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_x1 = torch.tensor(train_x1).to(torch.int64)
<ipython-input-25-3b33b724455d>:16: UserWarning: To copy construct from 

idx: 150 , loss: 1.3742, total loss: 422.234
idx: 300 , loss: 1.4101, total loss: 625.304
idx: 450 , loss: 1.1810, total loss: 808.322
idx: 600 , loss: 0.9938, total loss: 974.587
idx: 750 , loss: 0.6937, total loss: 1135.558
idx: 900 , loss: 1.0839, total loss: 1296.309
idx: 1050, loss: 1.2932, total loss: 1459.452
idx: 1200, loss: 0.9339, total loss: 1626.622
idx: 1350, loss: 1.0662, total loss: 1787.373
idx: 1500, loss: 0.7451, total loss: 1941.752
idx: 1650, loss: 1.1000, total loss: 2104.211
idx: 1800, loss: 0.7744, total loss: 2258.031
idx: 1950, loss: 0.8082, total loss: 2408.667
idx: 2100, loss: 0.9202, total loss: 2562.647
idx: 2250, loss: 1.2715, total loss: 2713.372
idx: 2400, loss: 0.8182, total loss: 2853.710
Epoch: 1
idx: 0   , loss: 0.6948, total loss: 2952.246
idx: 150 , loss: 1.2432, total loss: 3092.088
idx: 300 , loss: 1.4424, total loss: 3233.282
idx: 450 , loss: 1.0280, total loss: 3370.393
idx: 600 , loss: 0.6641, total loss: 3511.745
idx: 750 , loss: 1.1221, tota

In [26]:
def compute_loss(model, loader):
    model.eval()
    total_loss = 0.0
    for i, (x1, x2, y) in enumerate(testloader):
        with torch.no_grad():
            outputs = model.forward(x1, x2)
            loss = F.mse_loss(outputs, y)
            total_loss = total_loss + loss
    a = total_loss/(i)
    return a
if not skip_training:
    print(compute_loss(model, testloader)) # 0.8575

tensor(0.8628)


In [27]:
# Save the model to disk
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

Do you want to save the model (type yes to confirm)? yes
Model saved to recsys.pth.


In [28]:
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)